In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import cv2
import os
import time
import math
import main_functions as main

# SLM Linear

In [3]:
mnist_train = sio.loadmat('./mnist_train.mat')
mnist_test = sio.loadmat('./mnist_test.mat')
im_train, label_train = mnist_train['im_train'], mnist_train['label_train']
im_test, label_test = mnist_test['im_test'], mnist_test['label_test']
batch_size = 32
im_train, im_test = im_train / 255.0, im_test / 255.0

In [4]:
def get_mini_batch(im_train, label_train, batch_size):
    ## For ease of batching
    im_swp = np.swapaxes(im_train,0,1)
    lab_swp = np.swapaxes(label_train,0,1)
    
    ## One hot encoding
    oh_lab_swp = []
    for lab in lab_swp:
        oh = np.zeros(10)
        oh[lab] = 1
        oh_lab_swp.append(oh)
        
    np.random.shuffle(im_swp)
    np.random.shuffle(oh_lab_swp)
    
    mini_batch_x = np.array(np.array_split(im_swp,batch_size))#.reshape((-1,196,32))
    mini_batch_y = np.array(np.array_split(oh_lab_swp,batch_size))#.reshape((-1,10,32))
    
    return mini_batch_x,mini_batch_y

In [5]:
def fc(x, w, b):
    y = w @ x + b
    return y

In [6]:
def loss_euclidian(y_tilde, y):
    l = np.linalg.norm(y_tilde-y)**2
    dl_dy = 2 * (y_tilde-y)
    return l, dl_dy

In [7]:
def fc_backward(dl_dy, x, w, b, y):
    dl_dx = np.dot(w.T,dl_dy)
    dl_dw = np.outer(dl_dy,x)
    dl_db = y-dl_dy
    return dl_dx, dl_dw, dl_db

In [8]:
mini_batch_x,mini_batch_y = get_mini_batch(im_train, label_train, batch_size)

In [9]:
print(mini_batch_x.shape)
print(mini_batch_y.shape)

(32, 375, 196)
(32, 375, 10)


In [10]:
batches,imgs,size_img = mini_batch_x.shape
n = 10
m = size_img

In [11]:
lr = 0.1
dr = np.random.random(1)[0] + 1e-5
w = np.random.normal(0, 1, size=(n,m))
b = np.random.normal(0,0.1,size=n)
k = 0

In [12]:
for itr in range(4000):
    if itr % 1000 == 0:
        lr *= dr
        
    dl_dw = 0
    dl_db = 0
    
    for i,x in enumerate(mini_batch_x[k]):
        y = mini_batch_y[k][i]
        y_tilde = fc(x, w, b)
        l,dl_dy = loss_euclidian(y_tilde, y)
        dl_dx, dl_dw_n, dl_db_n = fc_backward(dl_dy, x, w, b, y)
        dl_dw += dl_dw_n
        dl_db += dl_db_n
        
    k += 1
    if k >= batches:
        k = 0
    w -= (lr * dl_dw)
    b -= (lr * dl_db)

<ipython-input-12-a35dc5722d1b>:14: RuntimeWarning: overflow encountered in add
  dl_db += dl_db_n
<ipython-input-12-a35dc5722d1b>:13: RuntimeWarning: overflow encountered in add
  dl_dw += dl_dw_n
<ipython-input-5-e90edc39b0c2>:2: RuntimeWarning: invalid value encountered in matmul
  y = w @ x + b


In [ ]:
print(w.shape)

In [14]:
mini_batch_x, mini_batch_y = get_mini_batch(im_train, label_train, batch_size)
w, b = train_slp_linear(mini_batch_x, mini_batch_y)
sio.savemat('slp_linear.mat', mdict={'w': w, 'b': b})

acc = 0
confusion = np.zeros((10, 10))
num_test = im_test.shape[1]
for i in range(num_test):
    x = im_test[:, [i]]
    y = fc(x, w, b)
    l_pred = np.argmax(y)
    confusion[l_pred, label_test[0, i]] = confusion[l_pred, label_test[0, i]] + 1

    if l_pred == label_test[0, i]:
        acc = acc + 1
accuracy = acc / num_test
for i in range(10):
    confusion[:, i] = confusion[:, i] / np.sum(confusion[:, i])

label_classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
visualize_confusion_matrix(confusion, accuracy, label_classes, 'Single-layer Linear Perceptron Confusion Matrix')

NameError: name 'train_slp_linear' is not defined